# Exercise III: Cart-Pole with TD(0)
>Solve the Cart-Pole control problem using TD(0) methods.


In [1]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import collections

In [2]:
#Cart-Pole Problem
env = gym.make("CartPole-v0")
env.reset()


for i in range(50000):
  action = env.action_space.sample()
  print("step i",i,"action=",action)
  obs, reward, done, info = env.step(action)
  print("obs=",obs,"reward=",reward,"done=",done,"info=",info)

  if done:
    break
    
env.close()
print("Iterations that were run:",i)

step i 0 action= 0
obs= [-0.03592544 -0.21272867  0.01448271  0.27566908] reward= 1.0 done= False info= {}
step i 1 action= 0
obs= [-0.04018002 -0.40805423  0.01999609  0.57288446] reward= 1.0 done= False info= {}
step i 2 action= 0
obs= [-0.0483411  -0.60345076  0.03145378  0.87179911] reward= 1.0 done= False info= {}
step i 3 action= 0
obs= [-0.06041012 -0.79898604  0.04888976  1.17420284] reward= 1.0 done= False info= {}
step i 4 action= 0
obs= [-0.07638984 -0.99470808  0.07237382  1.48180298] reward= 1.0 done= False info= {}
step i 5 action= 1
obs= [-0.096284   -0.80053984  0.10200988  1.2125724 ] reward= 1.0 done= False info= {}
step i 6 action= 0
obs= [-0.1122948  -0.99681963  0.12626132  1.53540046] reward= 1.0 done= False info= {}
step i 7 action= 1
obs= [-0.13223119 -0.80342403  0.15696933  1.2846386 ] reward= 1.0 done= False info= {}
step i 8 action= 0
obs= [-0.14829967 -1.00015689  0.18266211  1.62206951] reward= 1.0 done= False info= {}
step i 9 action= 1
obs= [-0.16830281 

## a. on-policy SARSA

## b. off-policy Q learning

## c. off-policy Expected SARSA with an epsilon-greedy policy